In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dash as dash
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
from dash import Dash

In [54]:
# Haremos lo mismo, pero por cantidad predicha
data_predicha = pd.read_csv('data_clean/data_con_predicciones.csv')

In [55]:
# importar los datos
df = pd.read_csv('data_clean/ventas_clean.csv')
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.sort_values('Fecha')
# Añadir columna 'Semana' relativa al primer día del dataset (Semana 1 = primeras 7 días)
df['Semana'] = ((df['Fecha'] - df['Fecha'].min()).dt.days // 7) + 1

In [56]:
productos_predichos = pd.read_csv('data_clean/data_con_predicciones_productos.csv')

In [62]:
# Vamos a crear una grafica interactiva para poder escoger entre regiones y categorias por semanas
app = Dash(__name__)
app.layout = html.Div([
    dcc.Dropdown(
        id='region-dropdown',
        options=[{'label': region, 'value': region} for region in df['Region'].unique()],
        value=df['Region'].unique().tolist(),
        multi=True
    ),
    dcc.Dropdown(
        id='categoria-dropdown',
        options=[{'label': categoria, 'value': categoria} for categoria in df['Categoria'].unique()],
        value=df['Categoria'].unique().tolist(),
        multi=True
    ),
    dcc.Graph(id='cantidad-graph')
])
@app.callback(
    Output('cantidad-graph', 'figure'),
    [Input('region-dropdown', 'value'),
     Input('categoria-dropdown', 'value')]
)
def update_graph(selected_regions, selected_categories):
    filtered_df = df[df['Region'].isin(selected_regions) & df['Categoria'].isin(selected_categories)].copy()
    categoria_tiempo_filtered = filtered_df.groupby(['Semana', 'Categoria'])['Cantidad'].sum().unstack().reset_index()
    categoria_tiempo_filtered = categoria_tiempo_filtered.sort_values('Semana')
    fig = px.bar(categoria_tiempo_filtered, x='Semana', y=categoria_tiempo_filtered.columns[1:],
                 title='Cantidad por Categoria a lo largo del tiempo por semanas',
                 labels={'value':'Cantidad', 'Semana':'Semana'})
    # Mostrar ticks en el eje x cada 2 semanas (ej. 1, 3, 5, ...)
    tick_start = int(categoria_tiempo_filtered['Semana'].min())
    tick_end = int(categoria_tiempo_filtered['Semana'].max())
    tickvals = list(range(tick_start, tick_end + 1, 2))
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=tickvals,
                                 ticktext=[str(v+4) for v in tickvals]))
    return fig
if __name__ == '__main__':
    app.run(debug=True)


In [63]:
# Dashboard: Cantidad Predicha por Semana — Productos (apilado)
productos_predichos = pd.read_csv('data_clean/data_con_predicciones_productos.csv')
productos_predichos['Fecha'] = pd.to_datetime(productos_predichos['Fecha'])
productos_predichos['Semana'] = ((productos_predichos['Fecha'] - productos_predichos['Fecha'].min()).dt.days // 7) + 1

# Cargar mapeo de productos
productos_map = pd.read_csv('data_clean/productos.csv')
productos_map = productos_map.rename(columns={'ID_Producto': 'ID_Producto', 'Nombre_producto': 'Nombre'})

# Merge para obtener nombres de productos
productos_predichos = productos_predichos.merge(productos_map[['ID_Producto', 'Nombre']], 
                                                on='ID_Producto', 
                                                how='left')

app_pred_productos = Dash(__name__)
app_pred_productos.layout = html.Div([
    html.H4('Cantidad Predicha Promedio por Semana — Productos (apilado)'),
    dcc.Dropdown(
        id='region-dropdown-pred-prod',
        options=[{'label': r, 'value': r} for r in sorted(productos_predichos['Region'].unique())],
        value=sorted(productos_predichos['Region'].unique()),
        multi=True,
        placeholder='Filtrar por Región'
    ),
    dcc.Graph(id='pred-productos-graph')
])

@app_pred_productos.callback(
    Output('pred-productos-graph', 'figure'),
    [Input('region-dropdown-pred-prod', 'value')]
)
def update_pred_productos_graph(selected_regions):
    if not selected_regions:
        selected_regions = productos_predichos['Region'].unique().tolist()
    
    filtered = productos_predichos[productos_predichos['Region'].isin(selected_regions)].copy()
    
    if filtered.empty:
        return px.bar(title='No hay datos para las selecciones')
    
    # Agrupar por Semana y Nombre (nombre del producto), promediando Cantidad_Semanal
    grouped_long = filtered.groupby(['Semana', 'Nombre'])['Cantidad_Semanal'].mean().reset_index().sort_values('Semana')
    
    # Gráfico apilado: productos por color, eje x = Semana, y = Cantidad_Semanal
    fig = px.bar(grouped_long, x='Semana', y='Cantidad_Semanal', color='Nombre',
                 title='Cantidad Promedio por Semana por Producto',
                 labels={'Cantidad_Semanal': 'Cantidad Promedio', 'Semana': 'Semana', 'Nombre': 'Producto'})
    
    # Mostrar ticks en el eje x cada 2 semanas
    tick_start = int(grouped_long['Semana'].min())
    tick_end = int(grouped_long['Semana'].max())
    tick_step = 2 if (tick_end - tick_start) >= 2 else 1
    tickvals = list(range(tick_start, tick_end + 1, tick_step))
    
    fig.update_layout(barmode='stack', xaxis=dict(tickmode='array', tickvals=tickvals, ticktext=[str(v+4) for v in tickvals]), xaxis_tickangle=-45, height=600)
    
    return fig

if __name__ == '__main__':
    app_pred_productos.run(debug=True)

In [64]:
# Dashboard: Cantidad Predicha por Categoria a lo largo del tiempo (semanal) para 2025
data_2025 = pd.read_csv('data_clean/data_2025_predicciones.csv')
data_2025['Fecha'] = pd.to_datetime(data_2025['Fecha'])
data_2025['Semana'] = ((data_2025['Fecha'] - data_2025['Fecha'].min()).dt.days // 7) + 1

app_2025 = Dash(__name__)
app_2025.layout = html.Div([
    dcc.Dropdown(
        id='region-dropdown-2025',
        options=[{'label': r, 'value': r} for r in sorted(data_2025['Region'].unique())],
        value=sorted(data_2025['Region'].unique()),
        multi=True
    ),
    dcc.Dropdown(
        id='categoria-dropdown-2025',
        options=[{'label': c, 'value': c} for c in sorted(data_2025['Categoria'].unique())],
        value=sorted(data_2025['Categoria'].unique()),
        multi=True
    ),
    dcc.Graph(id='cantidad-predicha-2025-graph')
])

@app_2025.callback(
    Output('cantidad-predicha-2025-graph', 'figure'),
    [Input('region-dropdown-2025', 'value'),
     Input('categoria-dropdown-2025', 'value')]
)
def update_2025_graph(selected_regions, selected_categories):
    if not selected_regions:
        selected_regions = data_2025['Region'].unique().tolist()
    if not selected_categories:
        selected_categories = data_2025['Categoria'].unique().tolist()

    filtered = data_2025[
        data_2025['Region'].isin(selected_regions) &
        data_2025['Categoria'].isin(selected_categories)
    ].dropna(subset=['Cantidad_Predicha']).copy()

    grouped = filtered.groupby(['Semana', 'Categoria'])['Cantidad_Predicha'].mean().unstack().reset_index().sort_values('Semana')

    if grouped.empty:
        return px.bar(title='No hay datos para las selecciones')

    fig = px.bar(grouped, x='Semana', y=grouped.columns[1:],
                 title='Cantidad Predicha Promedio por Categoria a lo largo del tiempo (2025)',
                 labels={'value': 'Cantidad Predicha Promedio', 'Semana': 'Semana'})

    tick_start = int(grouped['Semana'].min())
    tick_end = int(grouped['Semana'].max())
    tick_step = 2 if (tick_end - tick_start) >= 2 else 1
    tickvals = list(range(tick_start, tick_end + 1, tick_step))
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=tickvals, ticktext=[str(v+4) for v in tickvals]))

    return fig

if __name__ == '__main__':
    app_2025.run(debug=True)

In [60]:
# Dashboard: Cantidad de Productos Predichos en 2025
# Cargar la base de datos de predicciones de productos
productos_predichos_datos = pd.read_csv('data_clean/data_con_predicciones_productos.csv')
productos_predichos_datos['Fecha'] = pd.to_datetime(productos_predichos_datos['Fecha'])

# Cargar mapeo de productos para obtener nombres
productos_map = pd.read_csv('data_clean/productos.csv')
productos_map = productos_map.rename(columns={'ID_Producto': 'ID_Producto', 'Nombre_producto': 'Nombre'})

# Merge para obtener nombres de productos
productos_predichos_datos = productos_predichos_datos.merge(productos_map[['ID_Producto', 'Nombre']], 
                                                             on='ID_Producto', 
                                                             how='left')

# Crear semana
productos_predichos_datos['Semana'] = ((productos_predichos_datos['Fecha'] - productos_predichos_datos['Fecha'].min()).dt.days // 7) + 1

app_pred_prod_datos = Dash(__name__)
app_pred_prod_datos.layout = html.Div([
    html.H4('Cantidad de Productos Predichos en 2025'),
    dcc.Dropdown(
        id='region-dropdown-pred-prod-datos',
        options=[{'label': r, 'value': r} for r in sorted(productos_predichos_datos['Region'].unique())],
        value=sorted(productos_predichos_datos['Region'].unique()),
        multi=True,
        placeholder='Filtrar por Región'
    ),
    dcc.Graph(id='pred-productos-datos-graph')
])

@app_pred_prod_datos.callback(
    Output('pred-productos-datos-graph', 'figure'),
    [Input('region-dropdown-pred-prod-datos', 'value')]
)
def update_pred_productos_datos_graph(selected_regions):
    if not selected_regions:
        selected_regions = productos_predichos_datos['Region'].unique().tolist()
    
    filtered = productos_predichos_datos[productos_predichos_datos['Region'].isin(selected_regions)].copy()
    
    if filtered.empty:
        return px.bar(title='No hay datos para las selecciones')
    
    # Agrupar por Semana y Nombre (nombre del producto), promediando Cantidad_Semanal
    grouped_long = filtered.groupby(['Semana', 'Nombre'])['Cantidad_Semanal'].mean().reset_index().sort_values('Semana')
    
    # Gráfico apilado: productos por color, eje x = Semana, y = Cantidad_Semanal
    fig = px.bar(grouped_long, x='Semana', y='Cantidad_Semanal', color='Nombre',
                 title='Cantidad Promedio de Productos Predichos por Semana en 2025',
                 labels={'Cantidad_Semanal': 'Cantidad Promedio', 'Semana': 'Semana', 'Nombre': 'Producto'})
    
    # Mostrar ticks en el eje x cada 2 semanas
    tick_start = int(grouped_long['Semana'].min())
    tick_end = int(grouped_long['Semana'].max())
    tick_step = 2 if (tick_end - tick_start) >= 2 else 1
    tickvals = list(range(tick_start, tick_end + 1, tick_step))
    
    fig.update_layout(barmode='stack', xaxis=dict(tickmode='array', tickvals=tickvals, ticktext=[str(v+4) for v in tickvals]), xaxis_tickangle=-45, height=600)
    
    return fig

if __name__ == '__main__':
    app_pred_prod_datos.run(debug=True)

In [61]:
# Dashboard para cantidad predicha vs cantidad real (desde data_con_predicciones), podemos escoger region y categoria
data_predicha_final = data_predicha.copy()
data_predicha_final['Fecha'] = pd.to_datetime(data_predicha_final['Fecha'])
data_predicha_final['Semana'] = ((data_predicha_final['Fecha'] - data_predicha_final['Fecha'].min()).dt.days // 7) + 1

app_vs = Dash(__name__)
app_vs.layout = html.Div([
    dcc.Dropdown(
        id='region-dropdown-vs',
        options=[{'label': region, 'value': region} for region in sorted(data_predicha_final['Region'].unique())],
        value=sorted(data_predicha_final['Region'].unique()),
        multi=True
    ),
    dcc.Dropdown(
        id='categoria-dropdown-vs',
        options=[{'label': categoria, 'value': categoria} for categoria in sorted(data_predicha_final['Categoria'].unique())],
        value=sorted(data_predicha_final['Categoria'].unique()),
        multi=True
    ),
    dcc.Graph(id='cantidad-vs-predicha-graph')
])
@app_vs.callback(
    Output('cantidad-vs-predicha-graph', 'figure'),
    [Input('region-dropdown-vs', 'value'),
     Input('categoria-dropdown-vs', 'value')]
)
def update_graph_vs(selected_regions, selected_categories):
    if not selected_regions:
        selected_regions = data_predicha_final['Region'].unique().tolist()
    if not selected_categories:
        selected_categories = data_predicha_final['Categoria'].unique().tolist()
    
    filtered_df = data_predicha_final[data_predicha_final['Region'].isin(selected_regions) & data_predicha_final['Categoria'].isin(selected_categories)].copy()
    cantidad_tiempo = filtered_df.groupby(['Semana'])[['Cantidad_Semanal', 'Cantidad_Predicha']].sum().reset_index()
    cantidad_tiempo = cantidad_tiempo.sort_values('Semana')
    fig = px.line(cantidad_tiempo, x='Semana', y=['Cantidad_Semanal', 'Cantidad_Predicha'],
                  title='Cantidad real vs Cantidad predicha a lo largo del tiempo por semanas',
                  labels={'value':'Cantidad', 'Semana':'Semana'})
    # Mostrar ticks en el eje x cada 2 semanas (ej. 1, 3, 5, ...)
    tick_start = int(cantidad_tiempo['Semana'].min())
    tick_end = int(cantidad_tiempo['Semana'].max())
    tickvals = list(range(tick_start, tick_end + 1, 2))
    fig.update_layout(xaxis=dict(tickmode='array', tickvals=tickvals,
                                 ticktext=[str(v+4) for v in tickvals]))
    return fig
if __name__ == '__main__':
    app_vs.run(debug=True)